#### ERD Agent

In [2]:
#pip install openai python-dotenv

In [3]:
import os
from dotenv import find_dotenv
from dotenv import load_dotenv
from openai import OpenAI

In [5]:
env_file = find_dotenv('key.env')
load_dotenv(env_file)
api_key = os.getenv('OPENAI_API_KEY')
client = OpenAI(api_key=api_key)

In [7]:
pip install graphviz

Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install eralchemy[graphviz]

zsh:1: no matches found: eralchemy[graphviz]
Note: you may need to restart the kernel to use updated packages.


In [11]:
import tempfile
from eralchemy import render_er 

In [13]:
#generating the sql alchemy model by taking the user description and returning a response
def build_sqlalchemy_model(user_description):

    #complete sentences here to ensure llm outputs a complete model
    system_prompt = ("You are an expert at building SQLAlchemy models. "
    "Given a database description, write SQLAlchemy models using declarative_base. "
    "Only output the models. Do not include explanations, code fences, or comments.")

    user_input = f"Create SQLAlchemy models for this database: {user_description}"

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content":system_prompt},
            {"role": "user", "content": user_input}
        ],
        max_tokens = 800,
        temperature = 0.2
    )

    return response.choices[0].message.content

In [15]:
#saving the model to a temporary file
def save_model(erd_model):
    temp = tempfile.NamedTemporaryFile(delete=False, suffix = '.py', mode='w', encoding='utf-8')
    temp.write(erd_model)
    temp.close()
    return temp.name

In [17]:
def get_base_from_models(model_file_path):
    import importlib.util
    spec = importlib.util.spec_from_file_location("models", model_file_path)
    models = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(models)

    if hasattr(models, 'Base'):
        return models.Base
    else:
        raise ValueError("Could not find SQLAlchemy Base in generated models.")

In [21]:
if __name__=="__main__":
    print("Describe your database :")
    user_description = input("> ")
    print("\nGenerating SQLAlchemy models with GPT...")
    erd_model = build_sqlalchemy_model(user_description)
    print("\nGenerated SQLAlchemy models:\n")
    print(erd_model)
    print("\nSaving models and generating ERD image...")

    model_file_path = save_model(erd_model)
    base = get_base_from_models(model_file_path)
    output_file = "erd_output.png"
    render_er(base, output_file)
    print(f"\nERD image saved as {output_file} (open this file to view your diagram!)")    

Describe your database :


>  An online store with customers, products, categories, and orders. Each customer can place multiple orders. Each order can contain multiple products. Each product belongs to a category



Generating SQLAlchemy models with GPT...

Generated SQLAlchemy models:

from sqlalchemy import Column, Integer, String, ForeignKey, Table
from sqlalchemy.orm import relationship
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()

order_product_association = Table(
    'order_product', Base.metadata,
    Column('order_id', Integer, ForeignKey('orders.id'), primary_key=True),
    Column('product_id', Integer, ForeignKey('products.id'), primary_key=True)
)

class Customer(Base):
    __tablename__ = 'customers'
    
    id = Column(Integer, primary_key=True)
    name = Column(String, nullable=False)
    email = Column(String, unique=True, nullable=False)
    
    orders = relationship('Order', back_populates='customer')

class Product(Base):
    __tablename__ = 'products'
    
    id = Column(Integer, primary_key=True)
    name = Column(String, nullable=False)
    price = Column(Integer, nullable=False)
    category_id = Column(Integer, ForeignKey('categori